In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow import keras
from keras import models, layers, optimizers
from src.data_loader import X,Y 
from src.preprocessor import data_preprocessing

generator, val_data, test_data = data_preprocessing(X,Y) 
X_val, Y_val = val_data
X_test, Y_test = test_data


optimizer = keras.optimizers.Adam(learning_rate=0.001)


#definition of residual_unit
base_model = keras.applications.ResNet50(
    include_top = False, 
    weights="imagenet",
    input_shape= (224,224,3))

base_model.trainable = False 

#definition of classification head 
inputs = base_model.input
base_model_output = base_model.output
pooling = keras.layers.GlobalAveragePooling2D()(base_model_output )
dropout = keras.layers.Dropout(0.3)(pooling) 
outputs = keras.layers.Dense(101, activation="softmax")(dropout)

model = keras.models.Model(inputs=inputs, outputs=outputs)


model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(generator, epochs=10, validation_data = (X_val, Y_val) ) 

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

model.evaluate(X_test, Y_test)


model.summary()


plt.plot(train_acc, label="Trening")
plt.plot(val_acc, label = "Validation")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.title("Learning curve")
plt.legend()
plt.grid(True)
plt.show()

